# Notebook to test the reading of an AOR

In [127]:
from xml.etree.ElementTree import ElementTree as ET

aorfile = '../test/09_0158.aor'
tree = ET(file=aorfile)
vector = tree.find('list/vector')
# Extract Target and Instrument from each AOR
targets = [item.text for item in vector.findall('Request/target/name')]
instruments = [item.text for item in vector.findall('Request/instrument/data/InstrumentName')]

In [128]:
# Abstract
abstract = tree.find('list/ProposalInfo/ProposalAbstract')
print(abstract.text)

This proposal aims to characterize the properties of the neutral gas and HII regions in a sample of nearby, clumpy, turbulent galaxies. The properties of these galaxies most closely resemble those of z ~ 1 – 3 galaxies, when star formation activity was at its peak. The galaxies selected for this proposal are from the DYNAMO sample of galaxies from Green et al. (2010). Their disks have high internal velocity dispersions and high gas mass fractions much like z ~ 1 – 3 galaxies, and have star formation rates ranging from 10 to 80 solar masses per year. This gives us a unique opportunity to study the neutral interstellar medium in this turbulent, clumpy environment, without the difficulties that are encountered in high redshift studies. We will use FIFI-LS to observe the 157.7 μm [CII] and 88 μm [OIII]/63 μm [OI] fine-structure lines, and use HAWC+ observations in band C, D, and E to derive the total far-infrared luminosities and estimate the FIR dust temperatures. We will use the combinat

In [9]:
import numpy as np
instruments = np.array(instruments)
idx = instruments == 'FIFI-LS'

In [10]:
idx

array([ True,  True,  True,  True,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True])

# Flight plan table

In [4]:

import xml.etree.ElementTree as ET
aorfile = '../test/202105_FI_MAXINE_WX36.misxml'
tree = ET.parse(aorfile)
root = tree.getroot()

#root.tag
#root.attrib

for flightplan in root.iter('FlightPlan'):
    flightTime = flightplan.attrib['FltTime']
    flightName = flightplan.attrib['Id']
    departure = flightplan.attrib['DepartureTime']
    arrival = flightplan.attrib['ArrivalTime']
    print(flightName, flightTime, departure, arrival)

table = ''
fmt = '{0:s} & {1:s} & {2:s} & {3:s} & {4:s} & {5:s} & {6:s}\\\\\\n'
for leg in root.iter('Leg'):
    name = leg.find('Name').text
    start = leg.find('Start').text
    duration = leg.find('Duration').text
    altitude = leg.find('Alt').text
    planid = leg.find('ObsPlanID')
    if planid is None:
        planid = ''
    else:
        planid = str(planid.text)
        planid = planid.replace('_','\_')
        if planid is None:
            planid = ''
    elevation = leg.find('Elev')
    if elevation is not None:
        elstart = elevation.attrib['start']
        elend = elevation.attrib['end']
        elevation = elstart+'-'+elend
    else:
        elend = ''
        elstart=''
        elevation = ''
    ID = leg.attrib['id']
    ftype = leg.attrib['type']
    if ftype == 'Absolute':
        name = 'Climb'
    elif ftype == 'Arrival':
        name = 'Arrival'
    planid = planid.replace('None','')
    line = fmt.format(ID, name, planid, elevation, start, duration[0:5], altitude)
    print(line)
    
ID = planid = elevation = duration = ''
name = 'Landing'
start = arrival[11:19]
altitude = '0 ft'
line = fmt.format(ID, name, planid, elevation, start, duration, altitude)
print(line)
name = 'Total flight time'
start = altitude = ''
duration = flightTime
line = fmt.format(ID, name, planid, elevation, start, duration, altitude)
print(line)
    
    
# Save in a latex file - create a main.tex that includes it and run pdflatex

202105_FI_MAXINE 08:26:53 2021-06-05T03:45:00Z 2021-06-05T12:11:52Z
1 & Departure &  &  & 03:45:00 & 00:02 & 10000 ft\\\n
2 & Climb &  &  & 03:48:46 & 00:08 & 35000 ft\\\n
3 & Climb &  &  & 03:58:38 & 00:14 & 36000 ft\\\n
4 & Setup &  & 24.3-25.0 & 04:14:20 & 00:30 & 38000 ft\\\n
5 & m83-1 & 70\_0808 & 28.3-28.4 & 04:44:23 & 00:53 & 39000 ft\\\n
6 & IRAS 12112+0305 & 08\_0095 & 54.4-43.4 & 05:38:31 & 01:17 & 39000/35/41000 ft\\\n
7 & NGC6946_Nuc & 09\_0198 & 32.6-44.5 & 07:00:08 & 01:45 & 41000/75/43000 ft\\\n
8 & Sgr A & 71\_0024 & 30.3-31.1 & 08:51:17 & 00:54 & 43000 ft\\\n
9 & Mrk 0290 & 08\_0226 & 54.8-42.1 & 09:49:34 & 01:35 & 43000 ft\\\n
10 & Arrival &  &  & 11:26:29 & 00:45 & 43000 ft\\\n
 & Landing &  &  & 12:11:52 &  & 0 ft\\\n
 & Total flight time &  &  &  & 08:26:53 & \\\n


### Using the module

In [17]:
from fififly.flightplan import flightTable
aorfile = '../test/202105_FI_MAXINE_WX36.misxml'
table = flightTable(aorfile)
for t in table:
    print(t)


\begin{table}
\begin{center}
\begin{tabular}{rrccrrr}
\hline
\hline
Leg & Target & AOR_ID & Elevation & UTC start & Duration & Altitude\\
\hline
\hline
1  & Departure &  &  & 03:45:00 & 00:02 & 10000 ft\\
2  & Climb &  &  & 03:48:46 & 00:08 & 35000 ft\\
3  & Climb &  &  & 03:58:38 & 00:14 & 36000 ft\\
4  & Setup &  & 24.3-25.0 & 04:14:20 & 00:30 & 38000 ft\\
5  & m83-1 & 70_0808 & 28.3-28.4 & 04:44:23 & 00:53 & 39000 ft\\
6  & IRAS 12112+0305 & 08_0095 & 54.4-43.4 & 05:38:31 & 01:17 & 39000/35/41000 ft\\
7  & NGC6946_Nuc & 09_0198 & 32.6-44.5 & 07:00:08 & 01:45 & 41000/75/43000 ft\\
8  & Sgr A & 71_0024 & 30.3-31.1 & 08:51:17 & 00:54 & 43000 ft\\
9  & Mrk 0290 & 08_0226 & 54.8-42.1 & 09:49:34 & 01:35 & 43000 ft\\
10 & Arrival &  &  & 11:26:29 & 00:45 & 43000 ft\\
   & Landing &   &   & 12:11:52 &   & 0 ft\\
   & Total flight time &   &   &  & 08:26:53 & \\
\hline
\hline
\end{tabular}
\end{center}
\end{table}


In [148]:
legs = tree.find('FlightPlan/Legs')

## Table of legs

In [138]:
legs = [item for item in flightplan.findall('Legs')]

AttributeError: 'NoneType' object has no attribute 'findall'

TypeError: element indices must be integers

# Compile and display

In [16]:
import platform
import subprocess
print(platform.system().lower() )
output = subprocess.run(["ls", "-l", "."], capture_output=True)
subprocess.getoutput('ls ~')

darwin


'09_0038_backup.aor\nA85\nATT\nAdministration\nApplications\nBooks\nCIAO.ipynb\nCertificates\nChandra\nCodes\nCommunityDay\nDesktop\nDocuments\nDownloads\nFIFI-LS\nFORCAST\nHAWC\nHelp\nIDL\nIDLWorkspace\nIDLpipeline\nInterviews\nIrsLow\nLibrary\nModel\nMovies\nMusic\nNotes\nOLDPC\nOldDesktop\nPACS\nPapers\nPictures\nProposals\nPublic\nPython\nReferee\nReviews\nSciPy2019\nSites\nSpitzer\nTalks\nTeleTalk\nUsers\nWeb\nadam\nanaconda3\nbin\ncxcds_param4\ndemo.docx\nhcss\nmopex-script-env.bash\nmopex-script-env.csh\nold.spyder\nold.spyder-py3\nsofia'

In [ ]:
# works with Python3
import os
import platform
import subprocess

# TeX source filename
tex_filename = 'my_doc.tex'
filename, ext = os.path.splitext(tex_filename)
# the corresponding PDF filename
pdf_filename = filename + '.pdf'

# compile TeX file
subprocess.run(['pdflatex', '-interaction=nonstopmode', '-output-directory=pdf' tex_filename])

# check if PDF is successfully generated
if not os.path.exists(pdf_filename):
    raise RuntimeError('PDF output not found')

# open PDF with platform-specific command
if platform.system().lower() == 'darwin':
    subprocess.run(['open', pdf_filename])
elif platform.system().lower() == 'windows':
    os.startfile(pdf_filename)
elif platform.system().lower() == 'linux':
    subprocess.run(['xdg-open', pdf_filename])
else:
    raise RuntimeError('Unknown operating system "{}"'.format(platform.system()))


# Experiment

In [14]:
# Write a table in a file and run pdflatex

from fififly.flightplan import flightTable, parsTable, runLatex

aorfile = '../test/202105_FI_MAXINE_WX36.misxml'
mapfile = '../test/202105_FI_MAXINE_WX36.png'
flighttable = flightTable(aorfile)
filename = '../test/test.tex'
fifiparsfile = '../test/fifils-pars.json'
fifiparstable = parsTable(fifiparsfile)

with open(filename, 'w') as f:
    f.write(r'\documentclass{report}'+'\n')
    f.write(r'\topmargin      -10mm'+'\n')
    f.write(r'\oddsidemargin  -10mm'+'\n')
    f.write(r'\evensidemargin 17mm '+'\n')
    f.write(r'\textwidth      180mm'+'\n')
    f.write(r'\textheight     220mm'+'\n')
    f.write(r'\usepackage{graphicx}'+'\n')
    f.write(r'\usepackage{fancyhdr}'+'\n')
    f.write(r'\usepackage{lastpage}'+'\n')
    #f.write(r'\fancypagestyle{style1}{'+'\n')
    #f.write(r'\fancyhf{}'+'\n')
    #f.write(r'\fancyhead[R]{FIFI-LS Flight Description}'+'\n')
    ##f.write(r'\rfoot{Page \thepage / \pageref{LastPage}}'+'\n')
    #f.write(r'\fancyfoot[R]{\thepage / \pageref{LastPage}}'+'\n')
    #f.write(r'\renewcommand{\headrulewidth}{0.4pt}'+'\n')
    #f.write(r'}'+'\n')
    f.write(r'\renewcommand{\headrulewidth}{2pt}'+'\n')
    f.write(r'\pagestyle{fancy}'+'\n')
    f.write(r'\fancyhf{}'+'\n')
    f.write(r'\rhead{FIFI-LS Flight Description}'+'\n')
    f.write(r'\rfoot{Page \thepage / \pageref{LastPage}}'+'\n')
    f.write(r'\begin{document}'+'\n')
    #f.write(r'\large'+'\n')
    # Cover
    f.write(r'\begin{titlepage}'+'\n')
    f.write(r'\begin{center}'+'\n')
    f.write(r'\vspace*{1cm}'+'\n')
    f.write(r'\textbf{\Huge FIFI-LS\\}'+'\n')
    f.write(r'\vspace{0.5cm}'+'\n')
    f.write(r'{\huge Flight Description\\}'+'\n')
    f.write(r'\vspace{2.5cm}'+'\n')
    f.write(r'\includegraphics[width=0.5\textwidth]{../test/sofia}\\'+'\n')
    f.write(r'\vspace{2.5cm}'+'\n')
    f.write(r'\textbf{\Large Flight name: MAXINE}\\'+'\n')
    f.write(r'\vspace{0.5cm}'+'\n')
    f.write(r'\textbf{\Large SOFIA flight: 790}\\'+'\n')
    f.write(r'\vspace{0.5cm}'+'\n')
    f.write(r'\textbf{\Large FIFI-LS flight: 105}\\'+'\n')
    f.write(r'\vspace{0.5cm}'+'\n')
    f.write(r'\textbf{\Large Date: 2021-06-05}\\'+'\n')
    f.write(r'\vfill'+'\n')
    f.write(r'\end{center}'+'\n')
    f.write(r'\end{titlepage}'+'\n')
    # Definitions & Parameters
    #f.write(r'\lipsum[1-20]'+'\n')
    #f.write(r'\thispagestyle{style1}'+'\n')
    #f.write(r'\section{Definitions \& Parameters}'+'\n')
    f.write(r'\lhead{Definitions \& Parameters}'+'\n')
    # Insert Table
    for t in fifiparstable:
        f.write(t+'\n')
    #f.write(r'\newpage'+'\n')
    # Flight Plan
    #f.write(r'\lipsum[1-20]'+'\n')
    #f.write(r'\thispagestyle{style1}'+'\n')
    f.write(r'\lhead{Flight Plan}'+'\n')
    #f.write(r'\section{Flight Plan}'+'\n')
    # Insert Figure
    #f.write(r'\begin{figure}[ht]'+'\n')
    f.write(r'\begin{center}'+'\n')
    f.write(r'\includegraphics[width=0.75\textwidth]{'+mapfile+'}'+'\n')
    #f.write(r'\label{fig:flightmap}'+'\n')
    f.write(r'\end{center}'+'\n')
    #f.write(r'\end{figure}'+'\n')
    # Insert Table
    for t in flighttable:
        f.write(t+'\n')
    f.write(r'\end{document}')
    
runLatex(filename)

1  & Departure &  &  & 03:45:00 & 00:02 & 10000 ft\\
2  & Climb &  &  & 03:48:46 & 00:08 & 35000 ft\\
3  & Climb &  &  & 03:58:38 & 00:14 & 36000 ft\\
4  & Setup &  & 24.3-25.0 & 04:14:20 & 00:30 & 38000 ft\\
5  & m83-1 & 70\_0808 & 28.3-28.4 & 04:44:23 & 00:53 & 39000 ft\\
6  & IRAS 12112+0305 & 08\_0095 & 54.4-43.4 & 05:38:31 & 01:17 & 39000/35/41000 ft\\
7  & NGC6946\_Nuc & 09\_0198 & 32.6-44.5 & 07:00:08 & 01:45 & 41000/75/43000 ft\\
8  & Sgr A & 71\_0024 & 30.3-31.1 & 08:51:17 & 00:54 & 43000 ft\\
9  & Mrk 0290 & 08\_0226 & 54.8-42.1 & 09:49:34 & 01:35 & 43000 ft\\
10 & Arrival &  &  & 11:26:29 & 00:45 & 43000 ft\\
pdf file name  ../test/test.pdf
